<a href="https://colab.research.google.com/github/RICQW/pdf-input-url/blob/main/clean_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install camelot-py

In [4]:
import camelot

# 读取 PDF 表格
tables = camelot.read_pdf("https://cdn.jsdelivr.net/gh/RICQW/pdf-input-url/permitapplication(1).pdf", pages="all")

# 初始化 result
result = []

row_counter = 1  # 连续行编号
num_cols = 6     # 根据你的表格列数修改

# 1️⃣ 读取并清理内容
for table in tables:
    df = table.df
    for r in range(len(df)):
        for c in range(len(df.columns)):
            cleaned = df.iloc[r, c].replace(" ", "").replace("\n", "")
            cell_id = f"R{row_counter}C{c+1}"
            result.append((cell_id, cleaned))
        row_counter += 1

# 2️⃣ 按行分组
rows = [result[i:i+num_cols] for i in range(0, len(result), num_cols)]

# 3️⃣ 处理序号行：合并下一行内容到上一行
i = 1  # 从第二行开始，跳过第一行
while i < len(rows) - 1:  # 保证有下一行
    current_row = rows[i]
    if any(cell[1] == "序号" for cell in current_row):
        prev_row = rows[i-1]
        next_row = rows[i+1]
        for j in range(len(current_row)):
            prev_row[j] = (prev_row[j][0], prev_row[j][1] + next_row[j][1])
        # 删除序号行和下一行
        rows.pop(i+1)
        rows.pop(i)
        continue  # 不增加 i，下一行已经移位
    i += 1

# 4️⃣ 重新编号
final_result = []
row_counter = 1
for row in rows:
    for c_idx, cell in enumerate(row):
        cell_id = f"R{row_counter}C{c_idx+1}"
        final_result.append((cell_id, cell[1]))
    row_counter += 1

# 5️⃣ 输出最终结果
print(final_result)


[('R1C1', '序号'), ('R1C2', '类别'), ('R1C3', '记录内容'), ('R1C4', '记录频次'), ('R1C5', '记录形式'), ('R1C6', '其他信息'), ('R2C1', '1'), ('R2C2', '基本信息'), ('R2C3', '企业名称，生产经营场所地址、行业类别、法定代表人，统一社会信用代码、产品名称、生产工艺、生产规模、环保投资、排污权交易文件、环境影响评价审批意见及排污许可证编号等。'), ('R2C4', '对于未发生变化的基本信息，按年记录，1次/年；对于发生变化的基本信息，在发生变化时记录1次。'), ('R2C5', '电子台账+纸质台账'), ('R2C6', '至少保存五年'), ('R3C1', '2'), ('R3C2', '监测记录信息'), ('R3C3', '手工监测的记录采样记录：采样日期、采样时间、采样点位、混合取样的样品数量、采样器名称、采样人姓名等。样品保存和交接：样品保存方式、样品传输交接记录。样品分析记录：分析日期、样品处理方式、分析方法、质控措施、分析结果、分析人姓名等。自动监测运维记录包括自动监测系统运行状况、系统辅助设备运行状况、系统校准、校验工作等；仪器说明书及相关标准规范中规定的其它检查项目；校准、维护保养、维修记录等。'), ('R3C4', '监测数据的记录频次与标准规定的废气、废水监测频次一致。'), ('R3C5', '电子台账+纸质台账'), ('R3C6', '至少保存五年'), ('R4C1', '3'), ('R4C2', '其他环境管理信息'), ('R4C3', '排污单位在特殊时段应记录管理要求、执行情况（包括特殊时段生产设施运行管理信息和污染防治设施运行管理信息）。排污单位还应根据环境管理要求和排污单位自行监测内容需求，自行增补记录。'), ('R4C4', '重污染天气应对期间等特殊时段的台账记录频次原则上与正常生产记录频次一致，涉及特殊时段停产的排污单位或生产工序，该期间原则上仅对起始和结束当天进行1次记录。'), ('R4C5', '电子台账+纸质台账'), ('R4C6', '至少保存五年'), ('R5C1', '4'), ('R5C2', '生产设施运行管理信息'), ('R5C3', 'a汽车制造业排污单位应定期

In [5]:
import csv

with open("ledger(1).csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerows(final_result)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')